# Generating Qubit Hamiltonians

In [2]:
from utility import *

Specify the Qubit Hamiltonian of a molecule by its name, internuclear distances, basis set, and fermion-to-qubit transformation.
Here, we show the resulting Hamiltonian for $LiH$ in STO-3G with $1\overset{\circ}{A}$ between the $H$ and $Li$ atoms.

In [3]:
qubit_transf = 'jw' # Jordan-Wigner transformations
lih_H, lih_molecular_data = get_qubit_hamiltonian(mol='lih', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
lih_H

-3.934441956757915 [] +
-0.007923321157849954 [X0 X1 Y2 Y3] +
-0.0034145323580160323 [X0 X1 Y2 Z3 Z4 Y5] +
-0.0027468613203169424 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0034145323580160323 [X0 X1 X3 X4] +
-0.0027468613203169424 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.004864778381761675 [X0 X1 Y4 Y5] +
-0.0022963165872997085 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.002296316587299708 [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
-0.002472706168385288 [X0 X1 Y6 Y7] +
-0.0024727061683852886 [X0 X1 Y8 Y9] +
-0.0017744350099502914 [X0 X1 Y10 Y11] +
0.007923321157849954 [X0 Y1 Y2 X3] +
0.0034145323580160323 [X0 Y1 Y2 Z3 Z4 X5] +
0.0027468613203169424 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0034145323580160323 [X0 Y1 Y3 X4] +
-0.0027468613203169424 [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
0.004864778381761675 [X0 Y1 Y4 X5] +
0.0022963165872997085 [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.002296316587299708 [X0 Y1 Y5 Z6 Z7 Z8 Z9 X10] +
0.002472706168385288 [X0 Y1 Y6 X7] +
0.0024727061683852886 [X0 Y1 Y8 X9] +
0.

Alternatively, the qubit-tapering technique can find a smaller effective Hamitlonian by subsitituting operators with $\pm 1$. This technique is detailed in Bravyi's work ([Bravyi et al., "Tapering off qubits to simulate fermionic Hamiltonians", arXiv:1701.08213](https://arxiv.org/abs/1701.08213)). 

In [7]:
tapered_hamiltonian = taper_hamiltonian(lih_H, n_spin_orbitals=2 * lih_molecular_data.n_orbitals, n_electrons=lih_molecular_data.n_electrons, qubit_transf=qubit_transf)
print("The effective Hamiltonian:\n {}".format(tapered_hamiltonian))

6 4
The effective Hamiltonian:
 -3.7779692011782107 [] +
0.04069606505875718 [X0] +
-0.002366478148278278 [X0 X1 Y2 Y3] +
0.0072370922090284825 [X0 X1 Y2 Z3 Z6 Y7] +
0.0016354047160369379 [X0 X1 Z2 X3 Z4 Z5 Z7] +
-0.0031369437172317546 [X0 X1 Z2 Z4 Z5 Z6 X7] +
0.0072370922090284825 [X0 X1 X3 X6] +
-0.0016354047160369379 [X0 X1 X3 Z6 Z7] +
0.0067955266824246885 [X0 X1 X4] +
0.00679552668242469 [X0 X1 X5] +
-0.030603911418853515 [X0 X1 Y6 Y7] +
0.0031369437172317546 [X0 X1 X7] +
0.001635404716036938 [X0 Y1 Y2] +
0.002366478148278278 [X0 Y1 Y2 X3] +
-0.001635404716036938 [X0 Y1 Y2 Z3 Z4 Z5 Z7] +
-0.0072370922090284825 [X0 Y1 Y2 Z3 Z6 X7] +
0.0016354047160369379 [X0 Y1 Z2 Y3 Z4 Z5 Z7] +
-0.003136943717231754 [X0 Y1 Z2 Z3 Y6] +
0.003136943717231754 [X0 Y1 Z2 Z4 Z5 Y6 Z7] +
-0.0031369437172317546 [X0 Y1 Z2 Z4 Z5 Z6 Y7] +
0.0072370922090284825 [X0 Y1 Y3 X6] +
-0.0016354047160369379 [X0 Y1 Y3 Z6 Z7] +
0.0067955266824246885 [X0 Y1 Y4] +
0.00679552668242469 [X0 Y1 Y5] +
0.030603911418853515 [X0 

We can verify that this new Hamiltonian indeed includes the ground state. 

In [8]:
print("The ground state energy:")
obtain_PES('lih', [1], 'sto-3g', 'fci')

# Building the matrix representation of the effective Hamiltonian
# I, X, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]])
tapered_hamiltonian_tq = QubitHamiltonian.from_openfermion(tapered_hamiltonian)
lih_matrix = tapered_hamiltonian_tq.to_matrix()
# h2_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(lih_matrix)
print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))


The ground state energy:
E = -7.784460280267046 Eh

The eigenvalues in the effective Hamiltonian: 
 [-7.78446028 -7.65893236 -7.64449884 -7.29798386 -7.21203729 -7.14801009
 -7.08207994 -7.08207994 -7.06744927 -7.05640989 -7.05640989 -7.0349291
 -6.94675463 -6.93595983 -6.93595983 -6.83588218 -6.82201988 -6.82201988
 -6.72240847 -6.72240847 -6.66219376 -6.66219376 -6.5159243  -6.4546299
 -6.4546299  -6.41818695 -6.41818695 -6.24141804 -6.24141804 -5.74186898
 -5.7131875  -5.66538944 -5.65852466 -5.52263678 -5.4900516  -5.46853891
 -5.44221721 -5.40518529 -5.39065101 -5.37595514 -5.37595514 -5.37409494
 -5.34877816 -5.34877816 -5.33967037 -5.32223054 -5.31943314 -5.30094751
 -5.22313397 -5.21985321 -5.21985321 -5.21985321 -5.19489964 -5.19135211
 -5.17994973 -5.17866336 -5.16957466 -5.16957466 -5.16425403 -5.16425403
 -5.16306117 -5.15320069 -5.13184491 -5.10579765 -5.07762474 -5.02053953
 -5.02053953 -5.01000607 -5.00941829 -5.00941829 -4.99293519 -4.9818462
 -4.96400074 -4.94267868 -4

## $H_4$

In [18]:
h4, h4_molecular_data = get_qubit_hamiltonian(mol='h4', geometry=90, basis='sto3g', qubit_transf=qubit_transf)
print(f"Original hamiltonian: {h4}")
h4_tapered_hamiltonian = taper_hamiltonian(h4, n_spin_orbitals=2 * h4_molecular_data.n_orbitals, n_electrons=h4_molecular_data.n_electrons, qubit_transf=qubit_transf)
print("The effective Hamiltonian:\n {}".format(h4_tapered_hamiltonian))

print("The ground state energy:")
obtain_PES('h4', [90], 'sto-3g', 'fci')
h4_tapered_hamiltonian_tq = QubitHamiltonian.from_openfermion(h4_tapered_hamiltonian)
h4_matrix = h4_tapered_hamiltonian_tq.to_matrix()

eigvals, _ = np.linalg.eigh(h4_matrix)
print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

Original hamiltonian: -0.6427167034502669 [] +
-0.03382002547345963 [X0 X1 Y2 Y3] +
-0.033820025473459636 [X0 X1 Y4 Y5] +
-0.020901493208144092 [X0 X1 Y6 Y7] +
0.03382002547345963 [X0 Y1 Y2 X3] +
0.033820025473459636 [X0 Y1 Y4 X5] +
0.020901493208144092 [X0 Y1 Y6 X7] +
0.012952358031273271 [X0 Z1 X2 X4 Z5 X6] +
0.03426847228607794 [X0 Z1 X2 X5 Z6 X7] +
0.03426847228607794 [X0 Z1 X2 Y5 Z6 Y7] +
0.012952358031273264 [X0 Z1 Y2 Y4 Z5 X6] +
-0.03426847228607793 [X0 Z1 Z2 X3 Y4 Z5 Z6 Y7] +
-0.021316114254804668 [X0 Z1 Z2 X3 X5 X6] +
0.03426847228607793 [X0 Z1 Z2 Y3 Y4 Z5 Z6 X7] +
-0.021316114254804668 [X0 Z1 Z2 Y3 Y5 X6] +
0.03382002547345963 [Y0 X1 X2 Y3] +
0.033820025473459636 [Y0 X1 X4 Y5] +
0.020901493208144092 [Y0 X1 X6 Y7] +
-0.03382002547345963 [Y0 Y1 X2 X3] +
-0.033820025473459636 [Y0 Y1 X4 X5] +
-0.020901493208144092 [Y0 Y1 X6 X7] +
0.012952358031273264 [Y0 Z1 X2 X4 Z5 Y6] +
0.012952358031273271 [Y0 Z1 Y2 Y4 Z5 Y6] +
0.03426847228607794 [Y0 Z1 Y2 X5 Z6 X7] +
0.03426847228607794 [Y0 

## $NH_3$

In [ ]:
nh3, nh3_molecular_data = get_qubit_hamiltonian(mol='nh3', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(f"Original hamiltonian: {nh3}")
nh3_tapered_hamiltonian = taper_hamiltonian(nh3, n_spin_orbitals=2 * nh3_molecular_data.n_orbitals, n_electrons=nh3_molecular_data.n_electrons, qubit_transf=qubit_transf)
print("The effective Hamiltonian:\n {}".format(nh3_tapered_hamiltonian))

print("The ground state energy:")
obtain_PES('nh3', [1], 'sto-3g', 'fci')
nh3_tapered_hamiltonian_tq = QubitHamiltonian.from_openfermion(nh3_tapered_hamiltonian)
nh3_matrix = nh3_tapered_hamiltonian_tq.to_matrix()

eigvals, _ = np.linalg.eigh(nh3_matrix)
print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))



Original hamiltonian: -33.97122076932417 [] +
-0.011356296030625667 [X0 X1 Y2 Y3] +
0.003662369334943025 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Y9] +
-0.010106968035690814 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
0.003662369334943025 [X0 X1 X3 Z4 Z5 Z6 Z7 X8] +
-0.010106968035690814 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.002361470423547856 [X0 X1 Y4 Y5] +
-0.0024295451286337876 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Y13] +
-0.0024502524053377573 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Y15] +
-0.0024295451286337876 [X0 X1 X5 Z6 Z7 Z8 Z9 Z10 Z11 X12] +
-0.0024502524053377577 [X0 X1 X5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 X14] +
-0.0023614704235478457 [X0 X1 Y6 Y7] +
-0.002450252405337757 [X0 X1 Y6 Z7 Z8 Z9 Z10 Z11 Z12 Y13] +
0.002429545128633782 [X0 X1 Y6 Z7 Z8 Z9 Z10 Z11 Z12 Z13 Z14 Y15] +
-0.002450252405337757 [X0 X1 X7 Z8 Z9 Z10 Z11 X12] +
0.002429545128633782 [X0 X1 X7 Z8 Z9 Z10 Z11 Z12 Z13 X14] +
-0.006382564914551518 [X0 X1 Y8 Y9] +
0.0013025994835859798 [X0 X1 Y8 Z9 Z10 Y11] +
0.001302599483585979